In [1]:
# imports, settings and load data
import pandas as pd
from geopy.geocoders import Nominatim
import ipywidgets as widgets
from ipywidgets import fixed
import plotly.express as px
from matplotlib import gridspec
from matplotlib import pyplot as plt
import seaborn as sns
import time
from multiprocessing import Pool

geolocator = Nominatim(user_agent='questions')
data = pd.read_csv('./../datasets/kc_house_data.csv')

## 1. Qual a média dos preços de compra dos imóveis por "Nível"?
    - Nível 0 -> Preço entre R$ 0 e R$ 321.950
    - Nível 1 -> Preço entre R$ 321.950 e R$ 450.000
    - Nível 2 -> Preço entre R$ 450.000 e R$ 645.000
    - Nível 3 -> Preço acima de R$ 645.000

In [2]:
data['nivel'] = data['price'].apply(lambda x: 0 if x <= 321_950 else
                                              1 if 321_950 < x <= 450_000 else
                                              2 if 450_000 < x <= 645_000 else 3)

In [3]:
data[['nivel', 'price']].groupby('nivel').mean().reset_index()

,nivel,price
0,0,251557.649889
1,1,385688.681502
2,2,539730.960007
3,3,987540.224456


### R:
    - Nivel 0: média: R$ 251.557
- Nivel 1: média: R$ 385.688
    - Nivel 2: média: R$ 539.730
- Nivel 3: média: R$ 987.540

## 2. Qual a média do tamanho da sala de estar dos imóveis por "Size"?
    - Size 0 -> Tamanho entre 0 e 1427 sqft
    - Size 1 -> Tamanho entre 1427 e 1910 sqft    
    - Size 2 -> Tamanho entre 1910 e 2550 sqft    
    - Size 3 -> Tamanho acima de 2550 sqft    

In [4]:
data['size'] = data['sqft_living'].apply(lambda x: 0 if x <= 1427 else
                                                   1 if 1427 < x <= 1910 else
                                                   2 if 1910 < x <= 2550 else 3)

In [5]:
data[['size', 'sqft_living']].groupby('size').mean().reset_index()

,size,sqft_living
0,0,1123.832531
1,1,1664.962334
2,2,2211.792786
3,3,3329.607016


## R: 
    - Size 0: tamanho médio 1123 sqft
    - Size 1: tamanho médio 1664 sqft
    - Size 2: tamanho médio 2211 sqft
    - Size 3: tamanho médio 3329 sqft

## 3. Adicione as seguintes informações ao conjunto de dados original:
    - Place ID: Identificação de localização
    - OSM Type: Open Street Map type
    - Coutry: Nome do País
    - Country Code: Código do País 

In [6]:
data['query'] = data[['lat', 'long']].apply(lambda x: f"{x['lat']},{x['long']}", axis=1)

In [7]:
# def get_data(x):
#     index, row = x
#     time.sleep(3)
    
#     # API
#     response = geolocator.reverse(row['query'])
    
#     place_id = response.raw['place_id'] if 'place_id' in response.raw else 'NA'
#     osm_type = response.raw['osm_type'] if 'osm_type' in response.raw else 'NA'
#     country = response.raw['address']['county'] if 'county' in response.raw['address'] else 'NA'
#     country_code = response.raw['address']['country_code'] if 'country_code' in response.raw['address'] else 'NA'
   
#     return place_id, osm_type, country, country_code

In [8]:
import defs # File with the above function

df1 = data[['id', 'query']].head()

p = Pool(3)

start = time.process_time()
df1[['place_id', 'osm_type', 'country', 'country_code']] = p.map(defs.get_data, df1.iterrows())
end = time.process_time()

print(f"time Elapsed: {end - start}")


time Elapsed: 0.015625


In [9]:
df1

,id,query,place_id,osm_type,country,country_code
0,7129300520,"47.5112,-122.257",161022547,way,King County,us
1,6414100192,"47.721,-122.319",337363785,way,King County,us
2,5631500400,"47.7379,-122.233",74951055,node,King County,us
3,2487200875,"47.5208,-122.393",156726151,way,King County,us
4,1954400510,"47.6168,-122.045",296045104,way,King County,us


# 4. Aicione os seguintes filtros no Mapa:
    - Tamanho mínimo da área  da sala de estar.
    - Número mínimo de banheiros.
    - Valor máximo do Preço.
    - Tamanho máximo da área do porão.
    - Filtro das condições do imóvel.
    - Filtro por Ano de Construção.

In [10]:
# Button Size living room 
min_size_living_room = widgets.IntSlider(
    value = int(data['sqft_living'].mean()), 
    min = int(data['sqft_living'].min()),
    max = int(data['sqft_living'].max()),
    step = 1,
    description = 'Minimal living room area',
    disable = False, # False = Show button
    style = {'description_width' : 'initial'},
)

# Button Minimum number of bathrooms

min_number_bathrooms = widgets.IntSlider(
    value = int(data['bathrooms'].mean()),
    min = int(data['bathrooms'].min()),
    max = int(data['bathrooms'].max()),
    step = 1,
    description = 'Minimum number of bathrooms',
    disable = False,
    style = {'description_width' : 'initial'},
)

# Button Maximum Price Value

max_price = widgets.IntSlider(
    value = int(data['price'].mean()),
    min = int(data['price'].min()),
    max = int(data['price'].max()),
    step = 1,
    description = 'Maximum Price Value',
    disable = False,
    style = {'description_width' : 'initial'},
)

# Button Maximum size of basement area

max_basement_area = widgets.IntSlider(
    value = int(data['sqft_basement'].mean()),
    min = int(data['sqft_basement'].min()),
    max = int(data['sqft_basement'].max()),
    step = 1,
    description = 'Maximum size of basement area',
    disable = False,
    style = {'description_width' : 'initial'},
)

# Button Property condition filter
condition_limit = widgets.IntSlider(
    value = int(data['condition'].mean()),
    min = int(data['condition'].min()),
    max = int(data['condition'].max()),
    step = 1,
    description = 'Condition limit',
    disable = False,
    style = {'description_width' : 'initial'},
)

# Button Filter by Year of Construction
yr_built_limit = widgets.IntSlider(
    value = int(data['yr_built'].mean()),
    min = int(data['yr_built'].min()),
    max = int(data['yr_built'].max()),
    step = 1,
    description = 'Year Built',
    disable = False,
    style = {'description_width' : 'initial'},
)


In [11]:
def update_map(data, livingRoom, numberBathrooms, maxPrice, basementArea, condicao, built):
    df = data.loc[(data['sqft_living'] >= livingRoom) &
                  (data['bathrooms'] >= numberBathrooms) &
                  (data['price'] <= maxPrice) &
                  (data['sqft_basement'] <= basementArea) &
                  (data['condition'] == condicao) &
                  (data['yr_built'] >= built)]

    fig = px.scatter_mapbox(df,
                           lat='lat',
                           lon='long',
                           color='price',
                           size='price',
                           color_continuous_scale=px.colors.cyclical.IceFire,
                           size_max=15,
                           zoom=10)
    
    fig.update_layout(mapbox_style='open-street-map')
    fig.update_layout(height=600, margin={'r':0, 't':0, 'l':0, 'b':0})
    fig.show()

In [12]:
widgets.interactive(update_map, 
                    data = fixed(data), 
                    livingRoom = min_size_living_room, 
                    numberBathrooms = min_number_bathrooms,
                   maxPrice = max_price, 
                    basementArea = max_basement_area, 
                    condicao = condition_limit, 
                    built = yr_built_limit)

interactive(children=(IntSlider(value=2079, description='Minimal living room area', max=13540, min=290, style=…

## 5. Adicione os seguintes filtros no Dashboard
    - Filtro por data disponível para compra.
    - Variação do preço por dia.
    - Filtro se possui vista para a água ou não.

In [13]:
data['year'] = pd.to_datetime(data['date']).dt.strftime('%Y')
data['date'] = pd.to_datetime(data['date']).dt.strftime('%Y-%m-%d')
data['year_week'] = pd.to_datetime(data['date']).dt.strftime('%Y-%U')

In [14]:
date_limit = widgets.SelectionSlider(
    options = data['date'].sort_values().unique().tolist(),
    value = '2014-12-01',
    description = 'Max available date',
    continuous_update = False,
    style = {'description_width' : 'initial'},
    orientation = 'horizontal',
    readout = True
)

year_limit = widgets.SelectionSlider(
    options = data['yr_renovated'].sort_values().unique().tolist(),
    value = 2000,
    description = 'Max Year',
    continuous_update = False,
    style = {'description_width' : 'initial'},
    orientation = 'horizontal',
    readout = True
)

waterfront_limit = widgets.Checkbox(
    value=False,
    description='Is Waterfront?',
    disabled=False,
    indent=False
)



def update_map_dashboard(data, date_limit, year_limit, waterfront_limit):
    df = data[(data['date'] <= date_limit) &
             (data['yr_renovated'] >= year_limit) &
             (data['waterfront'] == waterfront_limit)].copy()
    
    
    fig = plt.figure(figsize=(24,12))
    specs = gridspec.GridSpec(ncols=2, nrows=2, figure=fig)
    
    ax1 = fig.add_subplot(specs[0, :])
    ax2 = fig.add_subplot(specs[1, 0])
    ax3 = fig.add_subplot(specs[1, 1])
    
    # First graph
    by_year = df[['price', 'year']].groupby('year').sum().reset_index()
    sns.barplot(x = 'year', y = 'price', data = by_year, ax = ax1)
    ax1.set_title('Sum Prices By Year')
    
    # Second graph
    by_day = df[['price', 'date']].groupby('date').mean().reset_index()
    sns.lineplot(x = 'date', y = 'price', data = by_day, ax = ax2)
    plt.xticks(rotation = 90)
    ax2.set_title('Price Change Per Day')
    
    # Third graph
    by_week_of_year = df[['price', 'year_week']].groupby('year_week').mean().reset_index()
    sns.barplot(x = 'year_week', y = 'price', data = by_week_of_year, ax = ax3)
    plt.xticks(rotation = 90)
    ax3.set_title('Average Price Per Week')

In [15]:
widgets.interactive(update_map_dashboard, 
                    data = fixed(data), 
                    date_limit = date_limit, 
                    year_limit = year_limit, 
                    waterfront_limit = waterfront_limit)

interactive(children=(SelectionSlider(continuous_update=False, description='Max available date', index=212, op…